In [5]:
import pandas as pd

# parquet 파일 경로 지정
file_path = 'drop.parquet'

# pyarrow(또는 fastparquet) 기반으로 parquet 파일 읽기
df = pd.read_parquet(file_path)

In [6]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,section_id,query_id,passage,question,answers_spans
0,revised,Answer the question using the information in t...,Answer the question using the information in t...,"[25, 25 yards, 25-yard]",[25],text,nfl_2036,d171f7e5-6fe9-4594-bb40-a7abb0808025,"Coming off their home win over the Browns, th...",How many yards was the second longest touchdow...,"{'spans': ['25', '25'], 'types': ['number', 'n..."
1,revised,Answer the question using the information in t...,Answer the question using the information in t...,"[lascarins, more lascarins]",[lascarins],text,history_1196,836c6420-a4ad-46ee-8dfa-0dc7f7e59ba9,"On 26 October 1591, the Portuguese, led by And...",In the first wave attack were there more Portu...,"{'spans': ['Lascarins', 'Lascarins'], 'types':..."
2,revised,Answer the question using the information in t...,Answer the question using the information in t...,"[verín, verin]",[verín],text,history_1284,c8d1b1f6-e810-4647-8a49-291d8a522a0a,Couceiro had undertaken a first incursion in 1...,"Where did Couceiro go first, Verin, or the bor...","{'spans': ['Verín', 'Verín'], 'types': ['span'..."
3,consensus,Answer the question using the information in t...,Answer the question using the information in t...,[738],[738],text,history_2847,0b8c86e3-8a03-4fb5-9e97-91ed7177bac1,"Puhl was granted free agent status in 1990, an...",How many runs and home runs combined did Puhl ...,"{'spans': ['738', '738'], 'types': ['number', ..."
4,rejected,Answer the question using the information in t...,Answer the question using the information in t...,None,[2200],text,history_200,f0177b59-9d57-4281-af99-363deb59de2f,"Charles reached Rome in 1265, but was temporar...",How many German mercenaries and Italian mercen...,"{'spans': ['2200', '2200'], 'types': ['number'..."


In [7]:
df['platinum_prompt'][0]

'Answer the question using the information in the context.\n\nContext: Coming off their home win over the Browns, the Steelers flew to Sun Life Stadium for a Week 7 intraconference duel with the Miami Dolphins.  Pittsburgh trailed in the first quarter as Dolphins kicker Dan Carpenter made a 39-yard and a 23-yard field goal.  The Steelers took the lead in the second quarter with a 22-yard field goal from kicker Jeff Reed, followed by quarterback Ben Roethlisberger hooking up with wide receiver Hines Ward on a 21-yard touchdown pass.  Miami answered with Carpenter getting a 22-yard field goal, yet Pittsburgh came right back as Roethlisberger found wide receiver Mike Wallace on a 53-yard touchdown pass.  The Dolphins would close out the half with quarterback Chad Henne completing a 25-yard touchdown pass to wide receiver Davone Bess. In the third quarter, the Steelers added onto their lead as Reed booted a 39-yard field goal.  Miami answered with Carpenter making a 37-yard field goal.  Mi

In [8]:
import pandas as pd
import re

# 1. Read the question again: ...
def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2'] = df['platinum_prompt'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Think step-by-step)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

# 2. Summarize the question first.
df['sum'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question first. ",
    regex=True
)

# 3. Organize the question into a table first.
df['table'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a table first. ",
    regex=True
)

# 4. Organize the question into a graph structure first.
df['graph'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Organize the question into a graph structure first. ",
    regex=True
)

# 5. Summarize the question in bullet points first.
df['bullet_point'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Summarize the question in bullet points first. ",
    regex=True
)

# 6. Read the question again first.
df['sRE2'] = df['platinum_prompt'].str.replace(
    r'(?=Think step-by-step)',
    "Read the question again first. ",
    regex=True
)

In [9]:
df['RE2'][0]

'Answer the question using the information in the context.\n\nContext: Coming off their home win over the Browns, the Steelers flew to Sun Life Stadium for a Week 7 intraconference duel with the Miami Dolphins.  Pittsburgh trailed in the first quarter as Dolphins kicker Dan Carpenter made a 39-yard and a 23-yard field goal.  The Steelers took the lead in the second quarter with a 22-yard field goal from kicker Jeff Reed, followed by quarterback Ben Roethlisberger hooking up with wide receiver Hines Ward on a 21-yard touchdown pass.  Miami answered with Carpenter getting a 22-yard field goal, yet Pittsburgh came right back as Roethlisberger found wide receiver Mike Wallace on a 53-yard touchdown pass.  The Dolphins would close out the half with quarterback Chad Henne completing a 25-yard touchdown pass to wide receiver Davone Bess. In the third quarter, the Steelers added onto their lead as Reed booted a 39-yard field goal.  Miami answered with Carpenter making a 37-yard field goal.  Mi

In [10]:
df['platinum_prompt_no_cot'][0]

'Answer the question using the information in the context.\n\nContext: Coming off their home win over the Browns, the Steelers flew to Sun Life Stadium for a Week 7 intraconference duel with the Miami Dolphins.  Pittsburgh trailed in the first quarter as Dolphins kicker Dan Carpenter made a 39-yard and a 23-yard field goal.  The Steelers took the lead in the second quarter with a 22-yard field goal from kicker Jeff Reed, followed by quarterback Ben Roethlisberger hooking up with wide receiver Hines Ward on a 21-yard touchdown pass.  Miami answered with Carpenter getting a 22-yard field goal, yet Pittsburgh came right back as Roethlisberger found wide receiver Mike Wallace on a 53-yard touchdown pass.  The Dolphins would close out the half with quarterback Chad Henne completing a 25-yard touchdown pass to wide receiver Davone Bess. In the third quarter, the Steelers added onto their lead as Reed booted a 39-yard field goal.  Miami answered with Carpenter making a 37-yard field goal.  Mi

In [11]:
import pandas as pd
import re

def repeat_question(match):
    # match.group(1): 첫 번째 줄바꿈 (예: "\n\n" 또는 "\n   \n")
    # match.group(2): 질문 텍스트
    # match.group(3): 두 번째 줄바꿈 (예: "\n\n")
    # 원하는 결과: 첫 번째 줄바꿈 뒤 기존 질문 대신 "Read the question again:" + 질문 텍스트를 두 번 반복하고,
    # 두 번째 줄바꿈을 그대로 붙인다.
    return f"{match.group(1)}{match.group(2)}\n\nRead the question again: {match.group(2)}{match.group(3)}"

# \n\n 사이에 공백이 있을 수도 있으므로 \n\s*\n 형태로 패턴을 수정
df['RE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(\n\s*\n)(.*?)(\n\s*\n)(?=Then, provide the final answer in the format)',
    repeat_question,
    flags=re.DOTALL,
    regex=True
)

df['sum_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Summarize the question first. ",
    regex=True
)

df['table_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Organize the question into a table first. ",
    regex=True
)

df['graph_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Organize the question into a graph structure first. ",
    regex=True
)

df['bullet_point_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Summarize the question in bullet points first. ",
    regex=True
)

df['sRE2_no_cot'] = df['platinum_prompt_no_cot'].str.replace(
    r'(?=Then, provide the final answer in the format)',
    "Read the question again first. ",
    regex=True
)

In [15]:
df['RE2_no_cot'][0]

'Answer the question using the information in the context.\n\nContext: Coming off their home win over the Browns, the Steelers flew to Sun Life Stadium for a Week 7 intraconference duel with the Miami Dolphins.  Pittsburgh trailed in the first quarter as Dolphins kicker Dan Carpenter made a 39-yard and a 23-yard field goal.  The Steelers took the lead in the second quarter with a 22-yard field goal from kicker Jeff Reed, followed by quarterback Ben Roethlisberger hooking up with wide receiver Hines Ward on a 21-yard touchdown pass.  Miami answered with Carpenter getting a 22-yard field goal, yet Pittsburgh came right back as Roethlisberger found wide receiver Mike Wallace on a 53-yard touchdown pass.  The Dolphins would close out the half with quarterback Chad Henne completing a 25-yard touchdown pass to wide receiver Davone Bess. In the third quarter, the Steelers added onto their lead as Reed booted a 39-yard field goal.  Miami answered with Carpenter making a 37-yard field goal.  Mi

In [13]:
df.to_parquet('drop_add_task.parquet')

In [14]:
df.head()

,cleaning_status,platinum_prompt,platinum_prompt_no_cot,platinum_target,original_target,platinum_parsing_strategy,section_id,query_id,passage,question,...,table,graph,bullet_point,sRE2,RE2_no_cot,sum_no_cot,table_no_cot,graph_no_cot,bullet_point_no_cot,sRE2_no_cot
0,revised,Answer the question using the information in t...,Answer the question using the information in t...,"[25, 25 yards, 25-yard]",[25],text,nfl_2036,d171f7e5-6fe9-4594-bb40-a7abb0808025,"Coming off their home win over the Browns, th...",How many yards was the second longest touchdow...,...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...
1,revised,Answer the question using the information in t...,Answer the question using the information in t...,"[lascarins, more lascarins]",[lascarins],text,history_1196,836c6420-a4ad-46ee-8dfa-0dc7f7e59ba9,"On 26 October 1591, the Portuguese, led by And...",In the first wave attack were there more Portu...,...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...
2,revised,Answer the question using the information in t...,Answer the question using the information in t...,"[verín, verin]",[verín],text,history_1284,c8d1b1f6-e810-4647-8a49-291d8a522a0a,Couceiro had undertaken a first incursion in 1...,"Where did Couceiro go first, Verin, or the bor...",...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...
3,consensus,Answer the question using the information in t...,Answer the question using the information in t...,[738],[738],text,history_2847,0b8c86e3-8a03-4fb5-9e97-91ed7177bac1,"Puhl was granted free agent status in 1990, an...",How many runs and home runs combined did Puhl ...,...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...
4,rejected,Answer the question using the information in t...,Answer the question using the information in t...,None,[2200],text,history_200,f0177b59-9d57-4281-af99-363deb59de2f,"Charles reached Rome in 1265, but was temporar...",How many German mercenaries and Italian mercen...,...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the question using the information in t...,Answer the 